<blockquote>
    <h1>Exercise 7.6</h1>
    <p>In this exercise, you will further analyze the <code>Wage</code> data set considered throughout this chapter.</p>
    <ol>
        <li>Perform polynomial regression to predict $\mathrm{wage}$ using $\mathrm{age}$. Use cross-validation to select the optimal degree $d$ for the polynomial. What degree was chosen, and how does this compare to the results of hypothesis testing using ANOVA? Make a plot of the resulting polynomial fit to the data.</li>
        <li>Fit a step function to predict $\mathrm{wage}$ using $\mathrm{age}$, and perform cross-validation to choose the optimal number of cuts. Make a plot of the fit obtained.</li>
    </ol>
</blockquote>